# TOR and Non-TOR

## All datasets are highly imbalanced
## For now only SelectedFeatures-10s-TOR-NonTOR dataset has been used.

#### Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score, precision_score, recall_score,roc_auc_score

#### Importing Dataset

In [2]:
data= pd.read_csv('C:\\Users\\AJIT\\Desktop\\Data science\\RP\\TOR\\Scenario-A\\SelectedFeatures-10s-TOR-NonTOR.csv')

C:\Users\AJIT\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.head()

,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,...,Bwd IAT Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label
0,10.0.2.15,53913,216.58.208.46,80,6,435,0,4597.7,435.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
1,10.0.2.15,53913,216.58.208.46,80,6,259,0,7722.01,259.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
2,10.0.2.15,53913,216.58.208.46,80,6,891,0,2244.67,891.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
3,10.0.2.15,53913,216.58.208.46,80,6,1074,0,1862.2,1074.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
4,10.0.2.15,53913,216.58.208.46,80,6,315,0,6349.21,315.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR


In [4]:
data.shape

(67834, 29)

#### Label Encoding for response variable

In [3]:
data['label'] = data['label'].map({'nonTOR': 1, 'TOR': 0})

#### Categorical Columns

In [4]:
cat_col = [c for i, c in enumerate(data.columns) if data.dtypes[i] in [np.object]]
cat_col 

['Source IP', ' Destination IP', ' Flow Bytes/s', ' Flow Packets/s']

#### 4 features has been excluded.

In [5]:
cols = [0,2,6,7]
data.drop(data.columns[cols],axis=1,inplace=True)

In [6]:
data.head()

,Source Port,Destination Port,Protocol,Flow Duration,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Mean,Fwd IAT Std,...,Bwd IAT Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label
0,53913,80,6,435,435.0,0.0,435,435,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
1,53913,80,6,259,259.0,0.0,259,259,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
2,53913,80,6,891,891.0,0.0,891,891,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
3,53913,80,6,1074,1074.0,0.0,1074,1074,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
4,53913,80,6,315,315.0,0.0,315,315,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1


#### Imbalanced data

In [13]:
data["label"].value_counts()

1    59790
0     8044
Name: label, dtype: int64

In [6]:
X = data.iloc[:, 1:24].values
y = data.iloc[:, 24].values

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Logistic Regression

In [17]:
# Baseline Model
from sklearn.metrics import f1_score, precision_score, recall_score,roc_auc_score
from sklearn.linear_model import LogisticRegression
lg=LogisticRegression()
lg.fit(X_train,y_train)
probs = lg.predict_proba(X_test)
probs = probs[:,1]
print('AUC:{}'.format(roc_auc_score(y_test, probs.round())))
print('Precision:{}'.format(precision_score(y_test, probs.round())))
print('Recall:{}'.format(recall_score(y_test, probs.round())))
print('F1 Score:{}'.format(f1_score(y_test, probs.round())))

C:\Users\AJIT\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


AUC:0.6877557722717059
Precision:0.925316973415133
Recall:0.9454241537818637
F1 Score:0.9352625051674246


In [18]:
#GridSearch for tuning using 7 CV
from sklearn.model_selection import GridSearchCV
lg1=LogisticRegression()
param_grid = {'C': [1.0]}
lg_gscv = GridSearchCV(lg1, param_grid, cv=7)
lg_gscv.fit(X_train, y_train)
probs = lg_gscv.predict_proba(X_test)
probs = probs[:,1]
print('AUC:{}'.format(roc_auc_score(y_test, probs.round())))
print('Precision:{}'.format(precision_score(y_test, probs.round())))
print('Recall:{}'.format(recall_score(y_test, probs.round())))
print('F1 Score:{}'.format(f1_score(y_test, probs.round())))

C:\Users\AJIT\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\AJIT\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\AJIT\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\AJIT\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\AJIT\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this w

AUC:0.6877557722717059
Precision:0.925316973415133
Recall:0.9454241537818637
F1 Score:0.9352625051674246


### KNN

In [19]:
#Baseline Model
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train,y_train)
probs = knn.predict_proba(X_test)
probs = probs[:,1]
print('AUC:{}'.format(roc_auc_score(y_test, probs.round())))
print('Precision:{}'.format(precision_score(y_test, probs.round())))
print('Recall:{}'.format(recall_score(y_test, probs.round())))
print('F1 Score:{}'.format(f1_score(y_test, probs.round())))

AUC:0.900392608904561
Precision:0.976240100041684
Recall:0.9786878395319683
F1 Score:0.9774624373956594


In [20]:
#GridSearch for tuning using 7 CV
from sklearn.model_selection import GridSearchCV
knn1 = KNeighborsClassifier()
param_grid = {'n_neighbors':[5,10]}
knn_gscv = GridSearchCV(knn1, param_grid, cv=7)
knn_gscv.fit(X_train, y_train)
probs = knn_gscv.predict_proba(X_test)
probs = probs[:,1]
print('AUC:{}'.format(roc_auc_score(y_test, probs.round())))
print('Precision:{}'.format(precision_score(y_test, probs.round())))
print('Recall:{}'.format(recall_score(y_test, probs.round())))
print('F1 Score:{}'.format(f1_score(y_test, probs.round())))

AUC:0.900392608904561
Precision:0.976240100041684
Recall:0.9786878395319683
F1 Score:0.9774624373956594


### SVM

In [ ]:
#Baseline Model
from sklearn.svm import SVC
svc = SVC(kernel='linear', probability=True)
svc.fit(X_train,y_train)
probs = svc.predict_proba(X_test)
probs = probs[:,1]
print('AUC:{}'.format(roc_auc_score(y_test, probs.round())))
print('Precision:{}'.format(precision_score(y_test, probs.round())))
print('Recall:{}'.format(recall_score(y_test, probs.round())))
print('F1 Score:{}'.format(f1_score(y_test, probs.round())))

In [ ]:
#GridSearch for tuning using 3 CV
from sklearn.model_selection import GridSearchCV
svc1 = SVC(kernel='linear', probability=True)
param_grid = {'C':[1]}
svc_gscv = GridSearchCV(svc1, param_grid, cv=3)
svc_gscv.fit(X_train, y_train)
probs = svc_gscv.predict_proba(X_test)
probs = probs[:,1]
print('AUC:{}'.format(roc_auc_score(y_test, probs.round())))
print('Precision:{}'.format(precision_score(y_test, probs.round())))
print('Recall:{}'.format(recall_score(y_test, probs.round())))
print('F1 Score:{}'.format(f1_score(y_test, probs.round())))

### Naive Bayes

In [11]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train,y_train)
probs = gnb.predict_proba(X_test)
probs = probs[:,1]
print('AUC:{}'.format(roc_auc_score(y_test, probs.round())))
print('Precision:{}'.format(precision_score(y_test, probs.round())))
print('Recall:{}'.format(recall_score(y_test, probs.round())))
print('F1 Score:{}'.format(f1_score(y_test, probs.round())))

AUC:0.8509739966704803
Precision:0.9870653095843935
Recall:0.7781027998328458
F1 Score:0.8702154507641258


In [12]:
from sklearn.model_selection import GridSearchCV
gnb1 = GaussianNB()
param_grid = {}
gnb_gscv = GridSearchCV(gnb1, param_grid, cv=7)
gnb_gscv.fit(X_train, y_train)
probs = gnb_gscv.predict_proba(X_test)
probs = probs[:,1]
print('AUC:{}'.format(roc_auc_score(y_test, probs.round())))
print('Precision:{}'.format(precision_score(y_test, probs.round())))
print('Recall:{}'.format(recall_score(y_test, probs.round())))
print('F1 Score:{}'.format(f1_score(y_test, probs.round())))

AUC:0.8509739966704803
Precision:0.9870653095843935
Recall:0.7781027998328458
F1 Score:0.8702154507641258


### Random Forest

In [13]:
random.seed(100)
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
probs = rf.predict_proba(X_test)
probs = probs[:,1]
print('AUC:{}'.format(roc_auc_score(y_test, probs.round())))
print('Precision:{}'.format(precision_score(y_test, probs.round())))
print('Recall:{}'.format(recall_score(y_test, probs.round())))
print('F1 Score:{}'.format(f1_score(y_test, probs.round())))

C:\Users\AJIT\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


AUC:0.9632415706860366
Precision:0.991
Recall:0.9938988717091517
F1 Score:0.992447319006885


In [14]:
rf.get_params

<bound method BaseEstimator.get_params of RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)>

In [15]:
from sklearn.model_selection import GridSearchCV
rf1 = RandomForestClassifier()
param_grid = {'n_estimators':[100,500,1000]}
rf_gscv = GridSearchCV(rf1, param_grid, cv=3)
rf_gscv.fit(X_train, y_train)
probs = rf_gscv.predict_proba(X_test)
probs = probs[:,1]
print('AUC:{}'.format(roc_auc_score(y_test, probs.round())))
print('Precision:{}'.format(precision_score(y_test, probs.round())))
print('Recall:{}'.format(recall_score(y_test, probs.round())))
print('F1 Score:{}'.format(f1_score(y_test, probs.round())))

AUC:0.9603960104194872
Precision:0.9899518352433151
Recall:0.996322607605516
F1 Score:0.9931270046236513


### Artificial Neural Network

In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense

C:\Users\AJIT\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [10]:
classifier = Sequential()

In [11]:
X_train.shape

(54267, 23)

In [12]:
classifier.add(Dense(units = 12, kernel_initializer = 'uniform', activation = 'relu', input_dim = 23))

In [13]:
classifier.add(Dense(units = 12, kernel_initializer = 'uniform', activation = 'relu'))

In [14]:
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

In [15]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [17]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 30)

Epoch 1/30
54267/54267 [==============================] - 8s 140us/step - loss: 1.8591 - acc: 0.8813
Epoch 2/30
54267/54267 [==============================] - 8s 139us/step - loss: 1.8596 - acc: 0.8813
Epoch 3/30
54267/54267 [==============================] - 8s 142us/step - loss: 1.8522 - acc: 0.8812
Epoch 4/30
54267/54267 [==============================] - 8s 139us/step - loss: 1.8650 - acc: 0.8813
Epoch 5/30
54267/54267 [==============================] - 8s 143us/step - loss: 1.7702 - acc: 0.8813
Epoch 6/30
54267/54267 [==============================] - 8s 141us/step - loss: 1.8412 - acc: 0.8813
Epoch 7/30
54267/54267 [==============================] - 8s 144us/step - loss: 1.8650 - acc: 0.8813
Epoch 8/30
54267/54267 [==============================] - 8s 142us/step - loss: 1.8652 - acc: 0.8813
Epoch 9/30
54267/54267 [==============================] - 8s 142us/step - loss: 2.0011 - acc: 0.8729
Epoch 10/30
54267/54267 [==============================] - 8s 141us/step - loss: 1.9154 - a

In [18]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [20]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [21]:
cm

array([[    0,  1602],
       [    0, 11965]], dtype=int64)